# Looker ERD Diagram for a given explore

Before running the script, follow the instructions at https://github.com/llooker/python_api_samples to configure a config file and save it in your working directory

Import packages

In [4]:
import lookerapi
import pandas as pd
import warnings
import numpy as np
from google.oauth2 import service_account
import pandas_gbq
import os
#warnings.filterwarnings("ignore")

Connect to your Looker Instance

In [5]:
#assumes that you the config.yml in current directory
config_filename = 'saleseng_config.yml'
looker = lookerapi.connect_looker(config_filename)

Enter a model name you want to create the ERD for

In [6]:
model_name = 'thelook_leigha'
model = looker.get_model(model_name) 

https://saleseng.dev.looker.com:19999/api/3.1/lookml_models/thelook_leigha


Iterate through all the explores and create a dataframe with the metadata for each field

In [7]:
field_dict = []
for explore_n in model['explores']:
    explore = looker.get_explore(explore_name=explore_n["name"],model_name=model_name)
    dimensions = explore['fields']['dimensions']
    for d in dimensions:
        d['explore'] = explore['label']
        d['explore_name'] = explore['name']
        d['model_name'] = model_name
        #d['explore_link'] = looker.host.replace(':19999/api/3.1/','')+'/explore/'+model_name+'/'+explore['name']
        d['explore_description'] = explore['description']
    measures = explore['fields']['measures']
    for m in measures:
        m['explore'] = explore['label']
        m['explore_name'] = explore['name']
        m['model_name'] = model_name
        #m['explore_link'] = looker.host.replace(':19999/api/3.1/','')+'/explore/'+model_name+'/'+explore['name']
        m['explore_description'] = explore['description']
    field_dict = field_dict+dimensions+measures

In [8]:
field_df = pd.DataFrame(field_dict)

Now we need to get the data in an appropriate text format for the command line tool https://github.com/BurntSushi/erd

In [9]:
explore_name = explore_n["name"] #enter explore you want to use
explore = looker.get_explore(explore_name=explore_name,model_name=model_name)

Initialize and fill the lists with info joins: the base table, joined in table and the details of the join

In [88]:
model_erd = dict()
model_erd_joins = dict()
model_erd_joins['base_tables'] = list()
model_erd_joins['secondary_tables'] = list()
model_erd_joins['relationships'] = list()
model_erd_joins['types'] = list()

In [89]:
model_erd[explore['name']] = dict()
#model_erd_joins['base_table'] = explore['name']
for i in range(len(explore['joins'])):
    model_erd[explore['joins'][i]['name']] = dict()
    #model_erd_joins[explore['joins'][i]['name']] = (explore['joins'][i]['relationship'], explore['joins'][i]['type'])
    
    tables_in_join = []
    for j in range(len(explore['joins'][i]['dependent_fields'])):
        field_name = explore['joins'][i]['dependent_fields'][j]
        tables_in_join.append(field_name[0:field_name.find('.')])
    tables_in_join = list(set(tables_in_join))

    model_erd_joins['base_tables'].append(tables_in_join[0])
    model_erd_joins['secondary_tables'].append(tables_in_join[1])
    model_erd_joins['relationships'].append(explore['joins'][i]['relationship'])
    model_erd_joins['types'].append(explore['joins'][i]['type'])

Function to transform the relationship name to proper cardinality format

In [103]:
def relationship_to_string(relationship):
    if relationship == 'many_to_one':
        return '*--1'
    elif relationship == 'one_to_one':
        return '1--1'
    elif relationship == 'many_to_many':
        return '*--*'
    else:
        return '1--*'

Add all the fields for each table and the type

In [104]:
for ix,i in enumerate(field_df['view']):
    for j in model_erd:
        if i == j:
            model_erd[j][field_df.iloc[ix]['field_group_variant']] = field_df.iloc[ix]['type']

Format text appropriately

In [134]:
erd_string = 'title {label: "Entity Relationship Diagram for %s Explore", size: "20"}' % explore['name']

erd_string += '\n\n # Entities \n'

for i in model_erd:
    #add table name
    erd_string += '\n[' + i + ']' 
    #add table fields and metadata
    for j in model_erd[i]:
        erd_string += '\n\t `%s` ' %j + '{label: "%s"}' %model_erd[i][j]
        
erd_string += '\n\n # Relationships \n\n'

for i in range(len(model_erd_joins['base_tables'])):
    erd_string += model_erd_joins['base_tables'][i] + ' '
    erd_string += relationship_to_string(model_erd_joins['relationships'][i])
    erd_string += ' ' + model_erd_joins['secondary_tables'][i] 
    erd_string += ' {label: "%s"}\n' % model_erd_joins['types'][i] 


Write the formatted string to a text file

In [139]:
model_text_file = open("model_erd.er","w+") 
model_text_file.write(erd_string)
model_text_file.close() 

Command line 'erd' function called to create the model_erd.pdf if the working directory

In [140]:
os.system("erd -i model_erd.er -o model_erd.pdf")

0

Print the string to actually see what it looks like

In [138]:
print(erd_string)

title {label: "Entity Relationship Diagram for inventory_snapshot Explore", size: "20"}

 # Entities 

[inventory_snapshot]
	 `Number In Stock` {label: "number"}
	 `Product ID` {label: "number"}
	 `Snapshot Date` {label: "date"}
	 `Stock Coverage Ratio` {label: "number"}
	 `Sum Stock Last Wk` {label: "sum"}
	 `Sum Stock Yesterday` {label: "sum"}
	 `Total In Stock` {label: "sum"}
	 `Stock Coverage Ratio Last Wk` {label: "number"}
	 `Stock Coverage Ratio Yday` {label: "number"}
	 `WoW Change - Coverage Ratio` {label: "string"}
[trailing_sales_snapshot]
	 `Product ID` {label: "number"}
	 `Snapshot Date` {label: "date"}
	 `Trailing 28d Sales` {label: "number"}
	 `Sum Trailing 28d Sales` {label: "sum"}
	 `Sum Trailing 28d Sales Last Wk` {label: "sum"}
	 `Sum Trailing 28d Sales Yesterday` {label: "sum"}
[products]
	 `Brand` {label: "string"}
	 `Category` {label: "string"}
	 `Department` {label: "string"}
	 `Distribution Center ID` {label: "number"}
	 `ID` {label: "number"}
	 `Item Name` {lab